In [27]:
import sys
sys.path.append('../')
import pytest
import xlwings as xw
import pandas as pd
from pylim import lim
pd.options.display.float_format = "{:,.2f}".format
from xlwings import Range, constants
from oilanalytics.freight import tce_calc, routes

In [28]:
wb = xw.Book(r'C:\dev\oilanalytics\notebooks\Baltic Exchange Tanker TCE Calculator SEP.20.xlsx')
sh = wb.sheets['TCE Calculator']

In [29]:
route = 'tc12'

In [30]:
df = routes.lim_data(route, start_date="date is within 1 days")
df = df.iloc[[-2]]
df

,MGO,VLSFO,HSFO,WorldScale,FlatRate
2021-10-06,92.67,577.27,498.99,125.63,18.74


In [31]:
ws = df.iloc[0].WorldScale
fr = df.iloc[0].FlatRate
mgo = df.iloc[0].MGO
vlsfo = df.iloc[0].VLSFO

In [32]:
routef = route.upper()
sh.range("b7").value = routef
sh.range("C7").value = ws
sh.range("D7").value = fr

In [33]:
sh.range("C14").value = vlsfo
# sh.range("C15").value = mgo

In [34]:
loadportcost = sh.range("c60").value
dischargeportcost = sh.range("c61").value

In [35]:
dd = {
    'FlatRate': fr,
    'WorldScale': ws,
    'MGO': mgo-5,
    'VLSFO': vlsfo-5,    
    'HSFO': vlsfo-5, 
    'LoadPortCosts': loadportcost,
    'DischargePortCosts': dischargeportcost
}
dd

{'FlatRate': 18.740000000000002,
 'WorldScale': 125.63,
 'MGO': 87.67,
 'VLSFO': 572.27,
 'HSFO': 572.27,
 'LoadPortCosts': 40545.0,
 'DischargePortCosts': 39769.76290097629}

In [36]:
ds = df.index[0]
data = pd.DataFrame(dd, index=[pd.to_datetime(ds)])
res = tce_calc.calc(route, data)
res

,FlatRate,WorldScale,MGO,VLSFO,HSFO,LoadPortCosts,DischargePortCosts,Freight_USDMT,GrossFreight,BunkerCost,NetFreight,TotalExpenses,NetIncome,TCE
2021-10-06,18.74,125.63,653.14,572.27,572.27,"40,545.00","39,769.76",23.54,"832,407.17","608,348.48","801,191.90","688,663.25","112,528.65","2,543.25"


In [37]:
cmp = sh.range("B79:C86").value
cmp = pd.DataFrame(cmp).set_index(0)
cmp = cmp.rename(columns={1:'Baltic'})
cmp['Code'] = pd.Series()
cmp.loc['Gross Freight', 'Code'] = res['GrossFreight'].iloc[0]
cmp.loc['Bunker Cost', 'Code'] = res['BunkerCost'].iloc[0]
cmp.loc['Net Income', 'Code'] = res['NetIncome'].iloc[0]
cmp.loc['Total Expenses', 'Code'] = res['TotalExpenses'].iloc[0]
cmp.loc['Baltic TCE', 'Code'] = res['TCE'].iloc[0]
cmp

C:\Users\ui906879\AppData\Local\Temp\1/ipykernel_27544/2024915855.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cmp['Code'] = pd.Series()


,Baltic,Code
0,,
Gross Freight,"832,407.17","832,407.17"
Net Freight,"801,191.90",NaN
Bunker Cost,"608,348.39","608,348.48"
Total Expenses,"688,663.15","688,663.25"
Net Income,"112,528.75","112,528.65"
- Gross,"3,248.74",NaN
- Net,"2,543.25",NaN
Baltic TCE,"2,543.00","2,543.25"


In [38]:
# gross freight
assert sh.range("C79").value == pytest.approx(res['GrossFreight'].iloc[0], abs=1e-1)

In [39]:
# bunker cost
assert sh.range("C81").value == pytest.approx(res['BunkerCost'].iloc[0], abs=1e-1)

In [40]:
# tce
assert sh.range("C86").value == pytest.approx(res['TCE'].iloc[0], abs=1e-1)

AssertionError: 

In [42]:
print("assert res['GrossFreight'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['GrossFreight'].iloc[0])
print("assert res['BunkerCost'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['BunkerCost'].iloc[0])
print("assert res['TCE'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['TCE'].iloc[0])

assert res['GrossFreight'][ds] ==  pytest.approx(832407.17, abs=1e-1)
assert res['BunkerCost'][ds] ==  pytest.approx(608348.48, abs=1e-1))
assert res['TCE'][ds] ==  pytest.approx(2543.25, abs=1e-1))
